In [1]:
import pandas as pd
from selection import ClassificationQualityComputer, GroundTruthQualityComputer, ClassificationCostComputer, CascadeRouter, Router, ConstantStrategy, GroundTruthQualityComputer, GroundTruthCostComputer, RepetitiveConstantStrategy
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression
from xgboost.sklearn import XGBClassifier, XGBRegressor
from sklearn.svm import SVC, SVR
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from tqdm import tqdm
from transformers import AutoTokenizer
from concurrent.futures import ThreadPoolExecutor, as_completed, ProcessPoolExecutor
from loguru import logger
import sys
import os
from collections import Counter

os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'

# set logger to only show info messages
logger.remove()
logger.add(sys.stdout, level="DEBUG")


/local/home/jdekoninck/miniconda3/envs/selection/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1

In [2]:
# dataset = 'mmlu'
# data_folder = '../data/classification'

# train_model_answers = pd.read_json(os.path.join(data_folder, dataset, 'train', 'model_answers.json'))
# train_costs = pd.read_json(os.path.join(data_folder, dataset, 'train', 'costs.json'))
# train_qualities = pd.read_json(os.path.join(data_folder, dataset, 'train', 'qualities.json'))
# train_queries = pd.read_json(os.path.join(data_folder, dataset, 'train', 'queries.json'))
# test_model_answers = pd.read_json(os.path.join(data_folder, dataset, 'test', 'model_answers.json'))
# test_costs = pd.read_json(os.path.join(data_folder, dataset, 'test', 'costs.json'))
# test_qualities = pd.read_json(os.path.join(data_folder, dataset, 'test', 'qualities.json'))
# test_queries = pd.read_json(os.path.join(data_folder, dataset, 'test', 'queries.json'))

# # train_queries = list(train_queries.apply(lambda x: x[0][1] + '\n' + x[1][1] + '\n' + x[2][1] + '\n' + x[3][1], axis=1))
# # test_queries = list(test_queries.apply(lambda x: x[0][1] + '\n' + x[1][1] + '\n' + x[2][1] + '\n' + x[3][1], axis=1))
# train_queries = list(train_queries[0])
# test_queries = list(test_queries[0])

In [3]:
from sklearn.model_selection import train_test_split
data = pd.read_csv('../data/routerbench_0shot.csv')

model_names = data.columns[3:14]

qualities = data[model_names]
cost_names = [model_name + '|total_cost' for model_name in model_names]
costs = data[cost_names]
# change column names
costs.columns = model_names
answer_names = [model_name + '|model_response' for model_name in model_names]
answers = data[answer_names]
# change column names
answers.columns = model_names

queries = np.array(data[['prompt']]).reshape(-1)

train_qualities, test_qualities, train_costs, test_costs, train_model_answers, test_model_answers, train_queries, test_queries = train_test_split(
                            qualities, costs, answers, queries, test_size=0.5, random_state=0                            
)

In [4]:
train_qualities_averaged = train_qualities.mean(axis=0)
train_costs_averaged = train_costs.mean(axis=0)

test_qualities_averaged = test_qualities.mean(axis=0)
test_costs_averaged = test_costs.mean(axis=0)

In [5]:
test_costs_averaged, test_qualities_averaged

(WizardLM/WizardLM-13B-V1.2           0.000072
 claude-instant-v1                    0.000231
 claude-v1                            0.002127
 claude-v2                            0.002400
 gpt-3.5-turbo-1106                   0.000241
 gpt-4-1106-preview                   0.003269
 meta/code-llama-instruct-34b-chat    0.000171
 meta/llama-2-70b-chat                0.000202
 mistralai/mistral-7b-chat            0.000045
 mistralai/mixtral-8x7b-chat          0.000134
 zero-one-ai/Yi-34B-Chat              0.000184
 dtype: float64,
 WizardLM/WizardLM-13B-V1.2           0.426736
 claude-instant-v1                    0.599162
 claude-v1                            0.632284
 claude-v2                            0.637035
 gpt-3.5-turbo-1106                   0.618305
 gpt-4-1106-preview                   0.779911
 meta/code-llama-instruct-34b-chat    0.197844
 meta/llama-2-70b-chat                0.330517
 mistralai/mistral-7b-chat            0.301375
 mistralai/mixtral-8x7b-chat          0.545

In [6]:
train_qualities_averaged, train_costs_averaged

(WizardLM/WizardLM-13B-V1.2           0.435519
 claude-instant-v1                    0.597665
 claude-v1                            0.627951
 claude-v2                            0.634576
 gpt-3.5-turbo-1106                   0.620336
 gpt-4-1106-preview                   0.782889
 meta/code-llama-instruct-34b-chat    0.206612
 meta/llama-2-70b-chat                0.326860
 mistralai/mistral-7b-chat            0.310848
 mistralai/mixtral-8x7b-chat          0.548849
 zero-one-ai/Yi-34B-Chat              0.652104
 dtype: float64,
 WizardLM/WizardLM-13B-V1.2           0.000073
 claude-instant-v1                    0.000235
 claude-v1                            0.002163
 claude-v2                            0.002438
 gpt-3.5-turbo-1106                   0.000245
 gpt-4-1106-preview                   0.003316
 meta/code-llama-instruct-34b-chat    0.000173
 meta/llama-2-70b-chat                0.000204
 mistralai/mistral-7b-chat            0.000046
 mistralai/mixtral-8x7b-chat          0.000

In [7]:
def convert_to_numpy(model_answers, costs, qualities, models, is_classification=True, is_routerbench=False):
    # if they are already numpy arrays, return them
    model_answers = model_answers[models].values
    if not is_routerbench:
        for i in range(len(model_answers)):
            for j in range(len(model_answers[i])):
                if is_classification:
                    model_answers[i, j] = np.array(model_answers[i, j])
                else:
                    model_answers[i, j][1] = np.array(model_answers[i, j][1])
    costs = costs[models].values
    qualities = qualities[models].values
    return model_answers, costs, qualities


In [8]:
def prediction(cascader, questions, qualities, costs, actual_answers, models, is_router=False):
    qualities_output = []
    costs_output = []
    models_run = []
    selected_models = []
    for i, question in enumerate(questions):
        model_answers = [[None for _ in range(len(qualities[i]))]]
        cost = 0
        models_run_question = []
        for step in range(len(model_answers[0])):
            model = cascader.predict([question], model_answers)
            if model[0] is None:
                break
            else:
                model_index = models.index(model[0])
                models_run_question.append(model_index)
                model_answers[0][model_index] = actual_answers[i][model_index]
                cost += costs[i][model_index]
            if is_router:
                break
        selected_answer = cascader.select_answer([question], model_answers)
        selected_model = models.index(selected_answer[0])
        quality = qualities[i][models.index(selected_answer[0])]
        qualities_output.append(quality)
        costs_output.append(cost)
        models_run.append(','.join([str(model) for model in models_run_question]))
        selected_models.append(selected_model)
    counter = Counter(models_run)
    counter_selected = Counter(selected_models)
    return {
        'quality': np.mean(qualities_output),
        'cost': np.mean(costs_output),
        'models_run': counter,
        'selected_models': counter_selected,
        'lambdas': list(cascader.get_lambdas())
    }

In [9]:
all_models = [
        {
            'name': 'meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo',
            'huggingface_name': 'meta-llama/Meta-Llama-3.1-8B-Instruct',
            'read_cost': 0.18 * 10 ** -6,
            'write_cost': 0.18 * 10 ** -6
        },
        {
            'name': 'meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo',
            'huggingface_name': 'meta-llama/Meta-Llama-3.1-70B-Instruct',
            'read_cost': 0.88 * 10 ** -6,
            'write_cost': 0.88 * 10 ** -6
        },
        {
            'name': 'meta-llama/Meta-Llama-3.1-405B-Instruct-Turbo',
            'huggingface_name': 'meta-llama/Meta-Llama-3.1-405B-Instruct',
            'read_cost': 5 * 10 ** -6,
            'write_cost': 5 * 10 ** -6
        },
        {
            'name': 'google/gemma-2-9b-it',
            'read_cost': 0.3 * 10 ** -6,
            'write_cost': 0.3 * 10 ** -6
        },
        {
            'name': 'google/gemma-2-27b-it',
            'read_cost': 0.8 * 10 ** -6,
            'write_cost': 0.8 * 10 ** -6
        },
        {
            'name': 'google/gemma-2b-it',
            'read_cost': 0.1 * 10 ** -6,
            'write_cost': 0.1 * 10 ** -6
        },
        {
            'name': 'mistralai/Mistral-7B-Instruct-v0.3',
            'read_cost': 0.2 * 10 ** -6,
            'write_cost': 0.2 * 10 ** -6
        },
        {
            'name': 'mistralai/Mixtral-8x22B-Instruct-v0.1',
            'read_cost': 1.2 * 10 ** -6,
            'write_cost': 1.2 * 10 ** -6
        },
        {
            'name': 'mistralai/Mixtral-8x7B-Instruct-v0.1',
            'read_cost': 0.6 * 10 ** -6,
            'write_cost': 0.6 * 10 ** -6
        },
    ]

In [10]:

class MinMaxLinearRegression(LinearRegression):
    def predict(self, X):
        return np.clip(super().predict(X), 0, 1)

def log_reg():
    return MinMaxLinearRegression()

In [11]:
# models = [all_models[0], all_models[3], all_models[1]]
# model_names_huggingface = [model.get('huggingface_name', model['name']) for model in models]
# tokenizers = [AutoTokenizer.from_pretrained(name) for name in model_names_huggingface]

# cost_computer = ClassificationCostComputer(
#         input_costs=[model['read_cost'] for model in models],
#         output_costs=[model['write_cost'] for model in models],
#         tokenizers=tokenizers,
#         constant_cost=False
#     )

cost_computer = GroundTruthCostComputer(0.0002, 0.00005, assume_constant=False)
# quality_computer = ClassificationQualityComputer(
#     model_class=lambda: log_reg(),
#     n_highest_include=2,
#     require_constant_not_run=False,
#     baseline=False,
#     include_question_embedding=False,
#     do_cosine_similarity=False,
#     is_regression=True,
#     all_model_combinations=True,
#     add_entropy=True,
#     add_equal_argmax=True,
#     add_js_divergence=True,
#     include_question_length=False,
#     store_all=True
# )
quality_computer = GroundTruthQualityComputer(0.6, 0.3)

train_split = 1800
model_names_here = model_names
model_names_here = sorted(model_names, key=lambda x: train_costs_averaged[x])
# model_names_here = [model['name'] for model in models]
train_model_answers_here, train_costs_here, train_qualities_here = convert_to_numpy(train_model_answers, train_costs, train_qualities, model_names_here, is_routerbench=False)
test_model_answers_here, test_costs_here, test_qualities_here = convert_to_numpy(test_model_answers, test_costs, test_qualities, model_names_here, is_routerbench=False)
# cost_computer.fit(train_queries[:train_split], train_model_answers_here[:train_split], train_costs_here[:train_split])
cost_computer.fit(
            np.concatenate([train_queries, test_queries], axis=0),
            np.concatenate([train_model_answers_here, test_model_answers_here], axis=0),
            np.concatenate([train_costs_here, test_costs_here], axis=0)
        )

# quality_computer.fit(train_queries[:train_split], train_model_answers_here[:train_split], train_qualities_here[:train_split])
quality_computer.fit(
            np.concatenate([train_queries, test_queries], axis=0),
            np.concatenate([train_model_answers_here, test_model_answers_here], axis=0),
            np.concatenate([train_qualities_here, test_qualities_here], axis=0)
        )

In [12]:
from tqdm import tqdm
from selection import BaselineCascader, HyperoptStrategy, RepetitiveConstantStrategy
import cProfile
router = CascadeRouter(
        cost_computer=cost_computer,
        quality_computer=quality_computer,
        models=model_names_here,
        max_expected_cost=0.003,
        strategies=[
            ConstantStrategy(10000, n_iterations=30),
            # HyperoptStrategy(10000, n_searches=500, max_factor=4),
        ],
        greedy=False,
        force_order=True,
        cascade=True,
        set_sigma_none=False
    )
# router = BaselineCascader(
#         cost_computer=cost_computer,
#         quality_computer=quality_computer,
#         models=model_names_here,
#         max_expected_cost=0.003,
#         strategies=[
#             ConstantStrategy(10000, n_iterations=30),
#             # HyperoptStrategy(10000, n_searches=500, max_factor=4),
#         ],
#     )


res = []
train_split=-2000
# cProfile.run('router.fit(train_queries[train_split:], train_model_answers_here[train_split:], train_qualities_here[train_split:], train_costs_here[train_split:])')
router.fit(
    train_queries[train_split:],
    train_model_answers_here[train_split:],
    train_qualities_here[train_split:],
    train_costs_here[train_split:]
)
# for lambda_ in tqdm(np.linspace(0, 1000, 20)):
#     router.lambda_ = lambda_
#     router.gamma = 0

#     start = 0
#     end = None
#     res.append(prediction(router, test_queries[start:end], test_qualities_here[start:end], test_costs_here[start:end], test_model_answers_here[start:end], model_names, 
#                           is_router=True))

2024-10-08 10:17:49.765 | DEBUG    | selection.lambda_strategy:compute_lambdas:45 - Computing lambdas with constant strategy


/local/home/jdekoninck/miniconda3/envs/selection/lib/python3.11/site-packages/scipy/stats/_qmc.py:958: UserWarning: The balance properties of Sobol' points require n to be a power of 2.
  sample = self._random(n, workers=workers)


2024-10-08 10:17:59.114 | DEBUG    | selection.lambda_strategy:compute_lambdas:66 - Lambda mid: 5000.0
2024-10-08 10:18:09.200 | DEBUG    | selection.lambda_strategy:compute_lambdas:75 - Cost: 6.594239999999824e-05
2024-10-08 10:18:09.204 | DEBUG    | selection.lambda_strategy:compute_lambdas:76 - Quality: 0.46942500000000004
2024-10-08 10:18:09.205 | DEBUG    | selection.lambda_strategy:compute_lambdas:66 - Lambda mid: 2500.0
2024-10-08 10:18:20.585 | DEBUG    | selection.lambda_strategy:compute_lambdas:75 - Cost: 9.721168999999555e-05
2024-10-08 10:18:20.589 | DEBUG    | selection.lambda_strategy:compute_lambdas:76 - Quality: 0.581225
2024-10-08 10:18:20.591 | DEBUG    | selection.lambda_strategy:compute_lambdas:66 - Lambda mid: 1250.0
2024-10-08 10:18:34.015 | DEBUG    | selection.lambda_strategy:compute_lambdas:75 - Cost: 0.00015693626399998964
2024-10-08 10:18:34.020 | DEBUG    | selection.lambda_strategy:compute_lambdas:76 - Quality: 0.684575
2024-10-08 10:18:34.021 | DEBUG    | 

In [13]:
prediction(router, test_queries, test_qualities_here, test_costs_here, test_model_answers_here, model_names_here, is_router=False)

{'quality': 0.8692942079018028,
 'cost': 0.0029516039132006697,
 'models_run': Counter({'0,1,2,3,4,5,6,7,8,9,10': 3248,
          '0,1,2,3,4,5,6,7': 3205,
          '0,1,2,3,4': 2312,
          '0,1,2,3,4,5,6,7,8': 2274,
          '0,1,2,3,4,5,6': 2167,
          '0,1,2,3,4,5,6,7,8,9': 1975,
          '0,1,2': 1533,
          '0,1': 797,
          '0,1,2,3,4,5': 386,
          '0': 321,
          '0,1,2,3': 31}),
 'selected_models': Counter({10: 3248,
          7: 3205,
          4: 2312,
          8: 2274,
          6: 2167,
          9: 1975,
          2: 1533,
          1: 797,
          5: 386,
          0: 321,
          3: 31}),
 'lambdas': [5.095256492495537,
  5.095256492495537,
  5.095256492495537,
  5.095256492495537,
  5.095256492495537,
  5.095256492495537,
  5.095256492495537,
  5.095256492495537,
  5.095256492495537,
  5.095256492495537,
  5.095256492495537]}

In [14]:
prediction(router, train_queries[train_split:], train_qualities_here[train_split:], train_costs_here[train_split:], 
           train_model_answers_here[train_split:], model_names_here, is_router=False)

{'quality': 0.87,
 'cost': 0.0030054042019999434,
 'models_run': Counter({'0,1,2,3,4,5,6,7,8,9,10': 355,
          '0,1,2,3,4,5,6,7': 328,
          '0,1,2,3,4': 253,
          '0,1,2,3,4,5,6,7,8': 252,
          '0,1,2,3,4,5,6': 225,
          '0,1,2,3,4,5,6,7,8,9': 212,
          '0,1,2': 176,
          '0,1': 94,
          '0': 53,
          '0,1,2,3,4,5': 46,
          '0,1,2,3': 6}),
 'selected_models': Counter({10: 355,
          7: 328,
          4: 253,
          8: 252,
          6: 225,
          9: 212,
          2: 176,
          1: 94,
          0: 53,
          5: 46,
          3: 6}),
 'lambdas': [5.095256492495537,
  5.095256492495537,
  5.095256492495537,
  5.095256492495537,
  5.095256492495537,
  5.095256492495537,
  5.095256492495537,
  5.095256492495537,
  5.095256492495537,
  5.095256492495537,
  5.095256492495537]}